In [ ]:
from hsmm_core.data_utils import DataLoader, TradingHours
from hsmm_core.feature_spaces import hmm_features
from hsmm_core.hsmm_runner import HmmCalibration
import time
from hsmm_core.consts import InitialisationMethod

n_hidden_states = 2

data_loader_init = {
    'trading_hours_filter': TradingHours.only_mkt_hours,
}


init_params = {
    'obs_model_name': 'ExpIndMixDiracGauss',
    'em_obs_init_method': InitialisationMethod.cluster,
    'em_hidden_init_method': InitialisationMethod.uniform,
    'no_hidden_states': n_hidden_states,
    'update_tag': 'tpsml'
}


ticker = 'BATS.L'

sd = '20180403'
ed = '20180419'

data_loader = DataLoader(**data_loader_init)

data_loader_hash = data_loader.data_loader_hash()

data = data_loader.load_trades_data(ticker, start_date=sd, end_date=ed)

hmm_calibration_engine = HmmCalibration(init_params=init_params)
hmm_calibration_engine.run_calibration_all_data(ticker, data,  data_loader_hash,
                                                force_recalc=True, use_multiprocessing=False,
                                                n_processes=2)




Number of points in data set is 22625, number of points with large price change 5117
Clustering of data produces highly unbalanced subsets; fall back to random intialisation.
Number of points in data set is 14535, number of points with large price change 3053
Clustering of data produces highly unbalanced subsets; fall back to random intialisation.
Number of points in data set is 13660, number of points with large price change 3720
Clustering of data produces highly unbalanced subsets; fall back to random intialisation.
Number of points in data set is 14830, number of points with large price change 4161
Clustering of data produces highly unbalanced subsets; fall back to random intialisation.
Number of points in data set is 22039, number of points with large price change 5061
Clustering of data produces highly unbalanced subsets; fall back to random intialisation.
Number of points in data set is 12028, number of points with large price change 3162
Clustering of data produces highly unbal

In [ ]:
features_engine = hmm_features()

s = time.time()
for date, date_data in data.iteritems():
    stored_hmm, _ = hmm_calibration_engine.get_calibrated_hmm(ticker, date, data_loader_hash)

    if stored_hmm is None:
       raise ValueError("No hmm available for date {}".format(date))

    features_engine.hmm = stored_hmm

    features = features_engine.generate_features(date_data)

In [ ]:
features